https://earthquake.usgs.gov/earthquakes/search/

In [4]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import pandas as pd

from parser.app import ClickHouseParser, CsvParser
from parser.db import client
from parser.repo import handler
from parser.repo.driver import create_tables, drop_tables

In [ ]:
year = 1990

start = datetime(year,3,6)
end = datetime(year,6,15)

parser = CsvParser(start, end, 'world', 'World_2000-2023')
parser.to_csv()

# Работа с таблицами

In [6]:
create_tables()

In [5]:
drop_tables('quakes_class')

In [7]:
result = handler.show_tables()
print(result)

ip_geo
quakes
quakes_class


# Загрузка данных в БД

In [ ]:
year = 2018
start = datetime(year,1, 1)
end = datetime(year, 6, 30)

parser = ClickHouseParser(start, end)

In [ ]:
parser.upload_to_bd()

In [ ]:
start = datetime(year, 7, 1)
end = datetime(year, 12, 31)
parser = ClickHouseParser(start, end)
parser.upload_to_bd()

In [ ]:
parser.update_bd()

In [ ]:
tmp_query = client.query_df("""
SELECT max(Date) as last
FROM quakes
""")

In [ ]:
last_datetime = tmp_query['last'].values[0].astype('M8[ms]').astype('O')
next_datetime = last_datetime + timedelta(0,1)
last_datetime, next_datetime

# Извлечение и анализ данных

In [ ]:
tmp_df = client.query_df('SELECT * FROM quakes ORDER BY Date')

In [ ]:
tmp_df.info()

In [ ]:
tmp_df.columns.to_list()[2:]

In [ ]:
spectr = tmp_df.columns.to_list()[2:]
dates = tmp_df['Date'].values.tolist()

plt.figure(figsize=(19, 5))
for col in spectr:
    plt.plot(dates, tmp_df[col].values.tolist())
plt.show()

In [ ]:
tmp_query = client.query_df("""
SELECT toDate(Date), count(toDate(Date)), max(Magnitude), min(Magnitude), divide(avg(Mercury_earth_distance), 30)
FROM quakes
GROUP BY toDate(Date)
ORDER BY toDate(Date)
""")

In [ ]:
max_mag = tmp_query['max(Magnitude)'].values.tolist()
min_mag = tmp_query['min(Magnitude)'].values.tolist()
merc =  tmp_query['divide(avg(Mercury_earth_distance), 30)'].values.tolist()
dates = tmp_query['toDate(Date)'].values.tolist()

plt.plot(dates, max_mag)
plt.plot(dates, min_mag)
plt.plot(dates, merc)
plt.show()

# Нормализация спектра с помощью функций

In [ ]:
from datafactory.normalize import msc, savgol, snv

In [ ]:
def graph_dataset(data_df: pd.DataFrame, snv_arr, msc_arr, savgol_arr):

    slice = 0, 1100
    chan = 0,150

    plt.figure(figsize=(19,8))

    plt.subplot(2,2,1)
    plt.plot(data_df.values[slice[0]:slice[1], chan[0]:chan[1]])
    plt.title('График исходных каналов')
    plt.subplot(2,2,2)
    plt.plot(snv_arr[slice[0]:slice[1],chan[0]:chan[1]])
    plt.title('График каналов после SNV нормализации')
    plt.subplot(2,2,3)
    plt.plot(msc_arr[slice[0]:slice[1], chan[0]:chan[1]])
    plt.title('График каналов после MSC нормализации')
    plt.subplot(2,2,4)
    plt.plot(savgol_arr[slice[0]:slice[1], chan[0]:chan[1]])
    plt.title('График каналов после сглаживания — фильтром Савицкого-Голея')

    plt.show()


snv_arr = snv(tmp_df.iloc[:, 4:].values)
msc_arr = msc(snv_arr)
savgol_arr = savgol(msc_arr)
graph_dataset(tmp_df.iloc[:, 4:], snv_arr, msc_arr, savgol_arr)